# We are going to use 10 lines of code using FeatureWiz and LazyTransform, two open source libraries to get a high score in Leaderboard.
### Currently our LB score on the Private LB is 13.02 which is just shy of the #1 ranked Private LB score of 12.58 - Not bad with just 10 lines of code.

<a href="https://github.com/AutoViML/featurewiz"><img src="https://i.ibb.co/ZLdZMZg/featurewiz-logos.png" alt="featurewiz-logos" border="0"></a>

In [ ]:
import pandas as pd
import numpy as np
import sklearn
sklearn.__version__

In [ ]:
!pip install featurewiz --ignore-installed --no-deps
!pip install xlrd --ignore-installed --no-deps

### You need to install this since Kaggle has a wrong version ##
!pip install Pillow==9.0.0

In [ ]:
!pip install lazytransform==0.73 --ignore-installed --no-deps

In [ ]:
import featurewiz as FW

In [ ]:
datapath = '../input/'
filename = "train.csv"
train = pd.read_csv(datapath+filename ,parse_dates=['date'])
filename = "test.csv"
test = pd.read_csv(datapath+filename, parse_dates=['date'])
test_ids = test['id'].values
test = test.drop('id', axis=1)
sample = pd.read_csv(datapath+'sample_submission.csv')
print(train.shape, test.shape)
train.head()

In [ ]:
target = 'sales'
modeltype = 'Regression'
ts_column = 'date'

# LazyTransform is an amazing library that converts categorical, NLP and Date-Time variables to Numeric in just one line of code. It works like a charm.

<a href="https://ibb.co/wyCGWJq"><img src="https://i.ibb.co/9n2KTw0/lazy-code.png" alt="lazy-code" border="0"></a>

In [ ]:
from lazytransform import LazyTransformer

In [ ]:
nunique_train=train.nunique().reset_index()
idvars = nunique_train[(nunique_train[0]==len(train)) | (nunique_train[0]==0) | (nunique_train[0]==1) ]['index'].tolist()
idvars

In [ ]:
### this is a list of variables in train that contain hierarchical info ###
hierarchical_vars=['store','item']

In [ ]:
####  Remember that target is a string variable but make sure 
####   it is a list by using targets variable ###
import copy
if isinstance(target, str):
    preds = [x for x in list(train) if x not in idvars+[target]]
    targets = [target]
else:
    preds = [x for x in list(train) if x not in idvars+target]
    targets = copy.deepcopy(target)
len(preds)

# We are Creating new features from hierarchical variables using the FeatureWiz library's groupby aggregator. This creates aggregate variables such as count, mean and max of the target column in train and transfers them to the test data. 

In [ ]:
### we will use the hierarchical columns as catcols in this aggregator 
print(len(hierarchical_vars))
### we will use the targets variable as numeric columns

In [ ]:
### You must send the entire train and test since target vars are involved
rcc = FW.Groupby_Aggregator(categoricals=hierarchical_vars,
                            aggregates=['count','mean','sum','max'], 
                            numerics=targets)
train = rcc.fit_transform(train)
test = rcc.transform(test)

In [ ]:
import copy
if isinstance(target, str):
    preds = [x for x in list(train) if x not in idvars+[target]]
    targets = [target]
else:
    preds = [x for x in list(train) if x not in idvars+target]
    targets = copy.deepcopy(target)
len(preds)

In [ ]:
X_train = train[preds]
y_train = train[target]
print(X_train.shape, y_train.shape)

In [ ]:
X_test = test[preds]
print(X_test.shape)

# Now we transform date-time features into additional features using LazyTransformer

In [ ]:
lazy = LazyTransformer(transform_target=False)
X_train, y_train = lazy.fit_transform(X_train, y_train)
print(X_train.shape)

In [ ]:
X_test = lazy.transform(X_test)
print(X_test.shape)

In [ ]:
if target in test.columns:
    y_test = test[target]

# We will also use a slick estimator known as SuloClassifier which has a flag to predict Integer Regression targets like the one we have here. It is also found in the featurewiz library.

<a href="https://github.com/AutoViML/featurewiz"><img src="https://i.ibb.co/zmGV4ct/Sulo-Classifier.png" alt="Sulo-Classifier" border="0"></a>

In [ ]:
from featurewiz import SuloClassifier, print_sulo_accuracy
spe = SuloClassifier(base_estimator=None, n_estimators=None, 
                     pipeline=True, regression=True, verbose=1)

In [ ]:
spe.fit(X_train, y_train)

# Now we can also see that the top features are date-time features which we created using LazyTransformer. We also see that the mean, max columns we created using featurewiz are also top features. This is very good news for us.

In [ ]:
spe.plot_importance()

In [ ]:
y_preds = spe.predict(X_test)
len(y_preds)

In [ ]:
y_probas = spe.predict_proba(X_test)

In [ ]:
if modeltype == 'Regression' and target in test.columns:
    from sklearn.metrics import r2_score, mean_squared_error
    def plot_scatter(y_test, y_preds):
        return pd.DataFrame(np.c_[y_test, y_preds], columns=['actuals','predictions']
                       ).plot(kind='scatter',x='actuals',y='predictions')
    print('R-Squared = %0.0f%%' %(100*r2_score(y_test,y_preds)))
    print('RMSE = %0.2f' %np.sqrt(mean_squared_error(y_test,y_preds)))
    plot_scatter(y_test, y_preds)


In [ ]:
subm = pd.DataFrame()
subm['id'] = test_ids
subm[target] = y_preds
subm.shape

In [ ]:
subm.head()

In [ ]:
subm.to_csv('submission.csv',index=False)

# We can see that this notebook gets a Private LB score of 13.02 with just 10 lines of code. If you can beat that score with fewer lines of code, then I'd like to know how you did it. If you liked this notebook and the ideas in it, please star the gihub library below
https://github.com/AutoViML